In [2]:
import sys
sys.path.insert(0, '5_uncertainty_testing/SBDynT/src')
import sbdynt as sbd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astroquery.jplsbdb import SBDB
from pathlib import Path

import rebound as rb
import celmech as cm
import assist

from linear_theory import linear_theory_prediction, make_simpler_secular_theory
from utils import ecliptic_to_icrf, icrf_to_ecliptic, ecliptic_xyz_to_elements

Searching NASA Horizons for 'sun'... 
Found: Sun (10) 


In [3]:
ephem = assist.Ephem("data/assist/linux_m13000p17000.441", "data/assist/sb441-n16.bsp")
nesvorny_data = pd.read_csv("data/nesvorny_catalog_dataset.csv")
prediction_path = Path("data/calc_propa")
prediction_path.mkdir(parents=True, exist_ok=True)

/tmp/ipykernel_3226708/3730445920.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  nesvorny_data = pd.read_csv("data/nesvorny_catalog_dataset.csv")


# Example Code for integrating backwards

In [4]:
# Get two example asteroids to test 
des = ['K15Bz3T', 'K08GG6Z']
epoch_n = 2460200.5 # Nesvorny epoch
epoch= 2460400.5 # the epoch your data is at

In [5]:
# list_of_sbodies = des
# ntp = len(list_of_sbodies)
# df1 = []
# (flag, x,y,z,vx,vy,vz) = sbd.query_sb_from_horizons(des=list_of_sbodies, epoch=epoch)
# if(flag):
# 	for n in range(0,ntp):
# 		df1.append({
# 			"Des'n": des[n],
# 			"epoch": epoch,
# 			"x": x[n], 
# 			"y": y[n], 
# 			"z": z[n], 
# 			"vx": vx[n]/365.25, 
# 			"vy": vy[n]/365.25, 
# 			"vz": vz[n]/365.25
# 		})
# df_e = pd.DataFrame(df1)
# df_e
sim = rb.Simulation()
sim.add(des[0], date = "JD%f"%epoch_n)
p = sim.particles[-1]
df1 = []
df1.append({
	"Des'n": des[0],
	"epoch": epoch,
	"x": p.x, 
	"y": p.y, 
	"z": p.z, 
	"vx": p.vx, 
	"vy": p.vy, 
	"vz": p.vz
})

df_e = pd.DataFrame(df1)
df_e

Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


,Des'n,epoch,x,y,z,vx,vy,vz
0,K15Bz3T,2460400.5,2.610227,1.173243,-0.453465,-0.214563,0.557543,0.020208


In [7]:
sim_a = rb.Simulation()
ex = assist.Extras(sim_a, ephem)
ex.gr_eih_sources = 11
sim_a.t = epoch - ephem.jd_ref
sim_a.ri_ias15.adaptive_mode = 2
sim_a.add("2024 YR4", plane = "frame", date="JD%f"%epoch)
p = sim.particles[-1]
print(p.xyz)
sim_a.move_to_com()
sim_a.exit_max_distance = 50.0
ex.integrate_or_interpolate(epoch_n - ephem.jd_ref)
p = sim.particles[-1]
print(p.xyz)

Searching NASA Horizons for '2024 YR4'... 
Found: (2024 YR4) 
[1.342139834380178, 2.6869603125265153, -0.2874707110867167]
[1.342139834380178, 2.6869603125265153, -0.2874707110867167]


In [ ]:
sim_a.add("2024 YR4", plane = "frame", date="JD%f"%epoch_n)
p = sim.particles[-1]
print(p.xyz)

Searching NASA Horizons for '2024 YR4'... 
Found: (2024 YR4) 
[1.342139834380178, 2.6869603125265153, -0.2874707110867167]


: 

In [6]:
# Convert orbital elements in xyz, vxvyvz
ast = df_e.iloc[0]
sim = rb.Simulation()
sim.add("Sun", date = "JD%f"%epoch)
sim.add(x = ast["x"].item(),
        y = ast["y"].item(),
        z = ast["z"].item(),
        vx = ast["vx"].item(),
    	vy = ast["vy"].item(),
        vz = ast["vz"].item(),
        date = "JD%f"%epoch)
p = sim.particles[1]

# Rotate to icrf
p = ecliptic_to_icrf(p)

# Use ASSIST to integrate backwards to Nesvorny epoch
sim_a = rb.Simulation()
ex = assist.Extras(sim_a, ephem)
ex.gr_eih_sources = 11
sim_a.t = epoch - ephem.jd_ref
sim_a.ri_ias15.adaptive_mode = 2
sim_a.add(p, plane = "frame", date="JD%f"%epoch)
sim_a.move_to_com()
sim_a.exit_max_distance = 50.0
ex.integrate_or_interpolate(epoch_n - ephem.jd_ref)
p = sim_a.particles[-1]
# print(p.xyz, p.vxyz)
# Rotate to ecliptic
p = icrf_to_ecliptic(p)
print(p.xyz, p.vxyz) # <---- I am correct until this point

# Convert it to orbital element (ASSIST)
# sim_a = rb.Simulation()
# sun_sim = rb.Simulation()
# sun_sim.add("sun", plane="frame", date="JD%f"%epoch_n)
# ex = assist.Extras(sim_a, ephem)
# sim_a.add(ast["Des'n"], plane = "frame", date="JD%f"%epoch_n)
# p_assist = sim_a.particles[-1]
# p_assist = icrf_to_ecliptic(p_assist)
# print(p_assist.xyz, p_assist.vxyz)

# orbit = p_assist.orbit(primary=sun_sim.particles[0])
# print(orbit)

# Convert it to orbital element (Rebound)
sim_a = rb.Simulation()
sim_a.add("sun", plane="ecliptic", date="JD%f"%epoch_n)
sim_a.add(ast["Des'n"], plane = "ecliptic", date="JD%f"%epoch_n)
p_rb = sim_a.particles[-1]
print(p_rb.xyz, p_rb.vxyz)
print(p_rb.orbit(primary = sim_a.particles[0]))
# print(p.orbit(primary = sim_a.particles[0]))
ecliptic_xyz_to_elements(p)

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
[2.6102270109946972, 1.1732433151634238, -0.4534650263530791] [-0.21456279796530978, 0.5575433657056174, 0.0202082823877012]
Searching NASA Horizons for 'sun'... 
Error: cannot interpolate beyond timestep bounds (h=1.945048e+01).


/home/lshen/workspace/rebound/rebound/simulation.py:927: RuntimeWarning: Particle is being adding from a simulation that uses different units.
  warnings.warn("Particle is being adding from a simulation that uses different units.", RuntimeWarning)


Found: Sun (10) 
Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 
[2.6102270109946972, 1.1732433151634238, -0.45346502635307906] [-0.21456279796530978, 0.5575433657056174, 0.020208282387701205]
<rebound.Orbit instance, a=3.0273227871350192 e=0.06284916803899557 inc=0.16222429427950347 Omega=1.7302078132465528 omega=4.042970809749725 f=0.9285876249598992>


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


<rebound.Orbit instance, a=3.0273227871350192 e=0.06284916803899557 inc=0.16222429427950347 Omega=1.7302078132465528 omega=4.042970809749725 f=0.9285876249598983>

In [21]:
nesvorny_data[nesvorny_data["Des'n"] == 'K15Bz3T'][["a", "e"]]

,a,e
901219,3.027323,0.062849


# Proper a integration

In [7]:
from multiprocessing import Pool
from pathlib import Path
from tqdm import tqdm
from itertools import islice

import pandas as pd
import numpy as np

import rebound as rb
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import assist
# %%
model_results = pd.read_csv("data/model_results.csv", index_col=0, dtype={"Des'n": str})

prediction_path = Path("data/calc_propa")
prediction_path.mkdir(parents=True, exist_ok=True)
ephem = assist.Ephem("data/assist/linux_m13000p17000.441", "data/assist/sb441-n16.bsp")

In [17]:
list = nesvorny_data[nesvorny_data["Des'n"] == 'K15Bz3T']
sim = rb.Simulation()
sim.add("Sun", date = "JD%f"%epoch)
sim.add(primary=sim.particles[0], a=list['a'].item(), e=list['e'].item(), inc=list['Incl.'].item()*np.pi/180, Omega=list['Node'].item()*np.pi/180, omega=list['Peri.'].item()*np.pi/180, M=list['M'].item()*np.pi/180)
sim.particles[1].xyz, sim.particles[1].vxyz

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 


([2.611176068218956, 1.1718093680587458, -0.4534739548411938], [-0.21446091756 ↪

↪ 484312, 0.5576385127912535, 0.020205606934606074])

In [21]:
object_name = des[0]
sim = rb.Simulation()
sim.t = epoch - ephem.jd_ref
sim.ri_ias15.adaptive_mode = 2
sim.add("Sun", date = "JD%f"%epoch)
sim.add(object_name, date="JD%f"%epoch)
sim.particles[1].xyz, sim.particles[1].vxyz

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 


([1.342139834380178, 2.6869603125265153, -0.2874707110867167], [-0.48893989550 ↪

↪ 407185, 0.29401127361598867, 0.07157873133342543])

In [12]:
def propa_calc(r):
	idx, row = r
	epoch = 2460200.5

	# Convert Nesvorny orbital elements into xyz, vxyz
	sim = rb.Simulation()
	sim.add("sun", date = "JD%f"%epoch, plane = "ecliptic")
	ps = sim.particles
	sim.add(primary=ps[0], 
		 a=row['a'], 
		 e=row['e'], 
		 inc=row['Incl.']*np.pi/180, 
		 Omega=row['Node']*np.pi/180, 
		 omega=row['Peri.']*np.pi/180, 
		 M=row['M']*np.pi/180)
	sim.add('K22SH4P', date = "JD%f"%epoch, plane = "ecliptic")
	print(sim.particles[-1].xyz, sim.particles[-2].xyz)
	p = ecliptic_to_icrf(sim.particles[-1])

	# ASSIST integration
	sim = rb.Simulation()
	ex = assist.Extras(sim, ephem)
	sim.t = epoch - ephem.jd_ref
	sim.ri_ias15.adaptive_mode = 2
	sim.add(p)
	print(sim.particles[-1].xyz, sim.particles[-1].vxyz)
	sim.add(row["Des'n"], date="JD%f"%epoch, plane = "frame")
	print(sim.particles[-1].xyz, sim.particles[-1].vxyz)
	sim.exit_max_distance = 50.0
	fpath = str(prediction_path / f"linear_prediction_results_{row["Des'n"]}.bin")
	sim.save_to_file(fpath, step=int(15*(np.pi*2)/sim.dt), delete_file=True)
	# sim.integrate(epoch + 1e4*np.pi*2, exact_finish_time=0)
	# result = get_simarchive_integration_results(str(prediction_path / f"linear_prediction_results_{row["Des'n"]}.bin"), coordinates='heliocentric')
	# print(result['a'].mean())
	# return row["Des'n"], result['a'].mean()

r = propa_calc(next(model_results.iterrows()))

Searching NASA Horizons for 'sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'K22SH4P'... 
Found: (2022 SP174) 
[-1.6097094694024572, 2.320448603850781, -0.005304456324078781] [-1.609707224360971, 2.3204653495904477, -0.005310022185252856]
[-1.6097094694024572, 2.1268599784542057, -0.9278881896110701] [-0.5280177761365163, -0.14629476535586466, 0.12907815509483067]
Searching NASA Horizons for 'K22SH4P'... 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)
/home/lshen/workspace/rebound/rebound/simulation.py:927: RuntimeWarning: Particle is being adding from a simulation that uses different units.
  warnings.warn("Particle is being adding from a simulation that uses different units.", RuntimeWarning)


Found: (2022 SP174) 
[-1.6097094694024572, 2.131079961536926, 0.9181547025995719] [-0.009083014032459303, -0.003340894018343768, -0.00031910930309418786]


In [67]:
list_of_sbodies = des
epoch= 2460300.5
ntp = len(list_of_sbodies)
df1 = []
(flag, x,y,z,vx,vy,vz) = sbd.query_sb_from_horizons(des=list_of_sbodies, epoch=epoch)
if(flag):
	for n in range(0,ntp):
		df1.append({
			"Des'n": des[n],
			"epoch": epoch,
			"x": x[n], 
			"y": y[n], 
			"z": z[n], 
			"vx": vx[n]/365.25, 
			"vy": vy[n]/365.25, 
			"vz": vz[n]/365.25
		})
df_e = pd.DataFrame(df1)
df_e

,Des'n,epoch,x,y,z,vx,vy,vz
0,K15Bz3T,2460300.5,2.102980,2.047809,-0.39294,-0.006491,0.007698,0.000850
1,K08GG6Z,2460300.5,-2.612442,0.843831,-0.72146,-0.002599,-0.010062,0.002199


In [38]:
nesvorny_data[nesvorny_data["Des'n"] == 'K15Bz3T'][["a", "e"]]

,a,e
901219,3.027323,0.062849


In [ ]:
list = nesvorny_data[nesvorny_data["Des'n"] == df_e["Des'n"]]
list_of_elements = ["a", "e", "Incl.", "Node", "Peri."]
elements = list[list_of_elements]
sim = rb.Simulation()
sim.add("Sun", date = "JD%f"%epoch)
sim.add(primary = sim.particles[0], a= elements["a"].item(), 
				e = elements["e"].item(), 
				inc = elements["Incl."].item()*np.pi/180, 
				Omega = elements["Node"].item()*np.pi/180, 
				omega = elements["Peri."].item()*np.pi/180)

<rebound.particle.Particle object at 0x7fd1f67a81d0, m=0.0 x=-2.2520454640988734 y=1.2072536966694403 z=-0.03397730387252601 vx=-0.2851673313763895 vy=-0.5223741274792107 vz=0.2636487915221468>

Code to convert xyz vxyz in default REBOUND simulation (ecliptic values) to orbital elements (dont' need. to update sun's mass from default REBOUND value)

In [ ]:
def integrate_nesvorny(r):
	idx, row = r
	# Convert orbital elements in xyz, vxvyvz
	list = nesvorny_data[nesvorny_data["Des'n"] == row["Des'n"]]
	elements = list[list_of_elements]
	sim = rb.Simulation()
	sim.add("Sun", date = "JD%f"%epoch)
	sim.add(primary=ps[0], a= elements["a"], 
				 e = elements["e"], 
				 inc = elements["Incl."]*np.pi/180, 
				 Omega = elements["Node"]*np.pi/180, 
				 omega = elements["Peri."]*np.pi/180)
	sim.add(ps)
	ps = sim.particles[1]

	# Rotate to icrf
	ps = ecliptic_to_icrf(ps)

	# Use ASSIST to integrate backwards to Nesvorny epoch
	sim_a = rb.Simulation()
	ex = assist.Extras(sim_a, ephem)
	sim_a.t = epoch - ephem.jd_ref
	sim_a.ri_ias15.adaptive_mode = 2
	sim_a.add(ps)
	sim_a.move_to_com()
	sim_a.exit_max_distance = 50.0
	sim_a.integrate(sim_a.t)
	p = sim_a.particle[-1]
	

In [ ]:
df_rb = []
def propa_calc(r):
	idx, row = r
	object_name = des[0]
	sim = rb.Simulation()
	sim.t = epoch - ephem.jd_ref
	sim.ri_ias15.adaptive_mode = 2
	sim.add("Sun", date = "JD%f"%epoch)
	sim.add(object_name, date="JD%f"%epoch)
	ps = sim.particles
	ast = ps[1].orbit(primary=ps[0])
	df_rb.append{
		"Des'n": object_name, 
		"a": ast.a,
		"e": ast.e,
		"Incl.": ast.inc,
		"Node": ast.Omega,
		"Peri.": ast.omega
	}

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'K15Bz3T'... 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


Found: 802375 (2015 BT613) 


In [11]:
ps[1].orbit(primary=ps[0]), ps[1].xyz

(<rebound.Orbit instance, a=3.0273227871350192 e=0.06284916803899557 inc=0.16222429427950347 Omega=1.7302078132465528 omega=4.042970809749725 f=0.9285876249598992>,
 [2.6102270109946972, 1.1732433151634238, -0.45346502635307906])

In [62]:
nesvorny_df = pd.read_csv("data/nesvorny_catalog_dataset.csv")
list = nesvorny_df[nesvorny_df["Des'n"] == des[0]]
list["Incl."] = list["Incl."] * np.pi/180
list[["e", "Incl.", "a", "Node", "Peri."]]

/tmp/ipykernel_3168533/877498960.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  nesvorny_df = pd.read_csv("data/nesvorny_catalog_dataset.csv")
/tmp/ipykernel_3168533/877498960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  list["Incl."] = list["Incl."] * np.pi/180


,e,Incl.,a,Node,Peri.
901219,0.062849,0.162224,3.027323,99.13362,231.64516


In [67]:
nesvorny_df = pd.read_csv("data/nesvorny_catalog_dataset.csv")
list = nesvorny_df[nesvorny_df["Des'n"] == des[0]]
nesvorny_df.columns

/tmp/ipykernel_3168533/3440952860.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  nesvorny_df = pd.read_csv("data/nesvorny_catalog_dataset.csv")


Index(['Unnamed: 0', 'Des'n', 'H_x', 'G', 'Epoch', 'M', 'Peri.', 'Node',
       'Incl.', 'e', 'n', 'a', 'propa', 'da', 'prope', 'de', 'propsini',
       'dsini', 'g', 's', 'H_y', 'NumOpps'],
      dtype='object')

In [74]:
list['Node'].item()

99.13362

Take Nesvorny elements and create a REBOUND simulation with right xyz vxyz in ecliptic frame at Nesvorny epoch

In [79]:
sim = rb.Simulation()
sim.add("Sun", date = "JD%f"%epoch)
sim.add(primary=ps[0], a=list['a'].item(), e=list['e'].item(), inc=list['Incl.'].item()*np.pi/180, Omega=list['Node'].item()*np.pi/180, omega=list['Peri.'].item()*np.pi/180, M=list['M'].item()*np.pi/180)
sim.particles[1].xyz, sim.particles[1].vxyz

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 


([2.610226919693984, 1.1732432674348257, -0.4534651707864257], [-0.21456279538 ↪

↪ 80095, 0.5575433854365816, 0.020208262787304556])

In [77]:
object_name = des[0]
sim = rb.Simulation()
sim.t = epoch - ephem.jd_ref
sim.ri_ias15.adaptive_mode = 2
sim.add("Sun", date = "JD%f"%epoch)
sim.add(object_name, date="JD%f"%epoch)
sim.particles[1].xyz, sim.particles[1].vxyz

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


([2.6102270109946972, 1.1732433151634238, -0.45346502635307906], [-0.214562797 ↪

↪ 96530978, 0.5575433657056174, 0.020208282387701205])

In [29]:
ps[1].orbit(primary=ps[0])

<rebound.Orbit instance, a=3.0273227871350197 e=0.06284916803899565 inc=0.41371352606488254 Omega=0.40788533938517807 omega=5.398500749226681 f=0.9285876249598983>

In [31]:
sim = rb.Simulation()
ex = assist.Extras(sim, ephem)
sim.t = epoch - ephem.jd_ref
sim.ri_ias15.adaptive_mode = 2
sim.add(des[0], date="JD%f"%epoch)

Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


In [32]:
sim.particles[-1]

<rebound.particle.Particle object at 0x7f3b9b409ad0, m=0.0 x=2.6102270109946972 y=1.2568077246023397 z=0.05064336166715787 vx=-0.0036909304815881177 vy=0.008661216289156868 vz=0.004133986932780778>

In [ ]:
df_icrf = []
for object_name in des:
	sim = rb.Simulation()
	ex = assist.Extras(sim, ephem)
	sim.t = epoch - ephem.jd_ref
	sim.ri_ias15.adaptive_mode = 2
	sim.add(object_name, date="JD%f"%epoch, plane = "frame")
	df_icrf.append({
		"Des'n": object_name,
		"x": sim.particles[-1].x, 
		"y": sim.particles[-1].y, 
		"z": sim.particles[-1].z
	})
df_icrf = pd.DataFrame(df_icrf)

Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 
Searching NASA Horizons for 'K08GG6Z'... 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


Found: (2008 GZ166) 


In [44]:
df_icrf

,Des'n,x,y,z
0,K15Bz3T,2.610227,1.256808,0.050643
1,K08GG6Z,-2.205842,1.984325,-0.112370


In [ ]:
df_e = []
for object_name in des:
	sim = rb.Simulation()
	ex = assist.Extras(sim, ephem)
	sim.t = epoch - ephem.jd_ref
	sim.ri_ias15.adaptive_mode = 2
	sim.add(object_name, date="JD%f"%epoch)
	df_e.append({
		"Des'n": object_name,
		"x": sim.particles[-1].x, 
		"y": sim.particles[-1].y, 
		"z": sim.particles[-1].z
	})
df_e = pd.DataFrame(df_e)

Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 
Searching NASA Horizons for 'K08GG6Z'... 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


Found: (2008 GZ166) 


In [46]:
df_e

,Des'n,x,y,z
0,K15Bz3T,2.610227,1.256808,0.050643
1,K08GG6Z,-2.205842,1.984325,-0.112370


In [ ]:
# Rotation matrix
deg_rot = -23.43929111 * np.pi / 180.0 # Ecliptic obliquity at J2000
matrix_data = [[1, 0, 0],
			   [0, np.cos(deg_rot), np.sin(deg_rot)],
			   [0, -np.sin(deg_rot), np.cos(deg_rot)]]
rot_mat = np.array(matrix_data)

In [47]:
coords = df_icrf[["x", "y", "z"]].values
df_icrf[["x", "y", "z"]] = coords @ rot_mat
# vecs = df_icrf[["vx", "vy", "vz"]].values
# df_icrf[["vx", "vy", "vz"]] = vecs @ rot_mat
df_icrf

,Des'n,x,y,z
0,K15Bz3T,2.610227,1.173243,-0.453465
1,K08GG6Z,-2.205842,1.775885,-0.892417


In [ ]:
list_of_sbodies = des
epoch= 2460200.5
ntp = len(list_of_sbodies)
df1 = []
(flag, x,y,z,vx,vy,vz) = sbd.query_sb_from_horizons(des=list_of_sbodies, epoch=epoch)
if(flag):
	for n in range(0,ntp):
		df1.append({
			"Des'n": des[n],
			"epoch": epoch,
			"x": x[n], 
			"y": y[n], 
			"z": z[n], 
			"vx": vx[n]/365.25, 
			"vy": vy[n]/365.25, 
			"vz": vz[n]/365.25
		})
df_e = pd.DataFrame(df1)
df_e

,Des'n,epoch,x,y,z,vx,vy,vz
0,K15Bz3T,2460200.5,2.618677,1.175319,-0.453679,-0.003695,0.009599,0.000348
1,K08GG6Z,2460200.5,-2.197392,1.777961,-0.892631,-0.005554,-0.008466,0.001211


In [33]:
coords = df_icrf[["x", "y", "z"]].values
df_icrf[["x", "y", "z"]] = coords @ rot_mat
# vecs = df_icrf[["vx", "vy", "vz"]].values
# df_icrf[["vx", "vy", "vz"]] = vecs @ rot_mat
df_icrf

,x,y,z
0,-2.366871,2.098494,-0.554786
1,1.214547,-1.942284,1.827072


In [30]:
df_e

,Des'n,epoch,x,y,z,vx,vy,vz
0,K15Bz3T,2460200.5,2.618677,1.175319,-0.453679,-0.003695,0.009599,0.000348
1,K08GG6Z,2460200.5,-2.197392,1.777961,-0.892631,-0.005554,-0.008466,0.001211


In [ ]:
# Combine all orbital elements into one df
['a', 'e', 'Incl.', 'Node', 'Peri.', 'M']
merged_data = {
	"Des'n": des, 
	"a": [1, 2], 
	"e": [1, 2], 
	"Incl.": [1, 2], 
	"Node.": [1, 2], 
	"Peri.": [1, 2], 
	"M": [1, 2]
}

array([2.61867662, 1.17531932])

In [ ]:
# Linear predict the proper orbital elements and secular frequencies
prediction_path = Path("data/linear_predictions")
prediction_path.mkdir(parents=True, exist_ok=True)

simpler_secular_theory = make_simpler_secular_theory()

def ecc_inc_prediction(r):
	idx, row = r
	sim = rb.Simulation('data/planets-epoch-2460200.5.bin')
	sim.add(
		a=row['a'],
		e=row['e'],
		inc=row['Incl.'] * np.pi / 180,
		Omega=row['Node'] * np.pi / 180,
		omega=row['Peri.'] * np.pi / 180,
		primary=sim.particles[0]
	)
	sim.particles[5].m
	for i in range(4): # this removes the four terrestrial planets
		sim.remove(index=1)
	sim.move_to_com()
	cm.nbody_simulation_utilities.align_simulation(sim)
	sim.integrator = 'whfast'
	sim.dt = np.min([p.P for p in sim.particles[1:]]) / 25.
	sim.ri_whfast.safe_mode = 0
	a = sim.particles[5].a
	e = sim.particles[5].e
	inc = sim.particles[5].inc
	omega = sim.particles[5].omega
	Omega = sim.particles[5].Omega

	u0, v0, g0, s0 = linear_theory_prediction(e, inc, omega, Omega, a, row['propa'], simpler_secular_theory)

	np.savez(prediction_path / f"linear_prediction_results_{row["Des'n"]}", u=u0, v=v0, g=g0, s=s0)
# %%
with Pool(40) as p:
	table = p.map(ecc_inc_prediction, nesvorny_df.iterrows())


In [ ]:
# Rotation matrix
deg_rot = -23.43929111 * np.pi / 180.0 # Ecliptic obliquity at J2000
matrix_data = [[1, 0, 0],
			   [0, np.cos(deg_rot), np.sin(deg_rot)],
			   [0, -np.sin(deg_rot), np.cos(deg_rot)]]
rot_mat = np.array(matrix_data)
vec_1_trans = rot_mat @ vec_1[:3]
print(vec_1_trans)
vec_2_trans = rot_mat @ vec_2[:3]
print(vec_2_trans)

[2.61867662 1.25879761 0.05127268]
[-2.19739233  1.98631512 -0.11174071]


In [ ]:
df_icrf = []
for object_name in list_of_sbodies:
	sim = rb.Simulation()
	ex = assist.Extras(sim, ephem)
	sim.t = epoch - ephem.jd_ref
	sim.ri_ias15.adaptive_mode = 2
	sim.add(object_name,plane="frame", date="JD%f"%epoch)
	# print(sim.particles[-1])
	x = sim.particles[-1].x
	y = sim.particles[-1].y
	z = sim.particles[-1].z
	vx = sim.particles[-1].vx
	vy = sim.particles[-1].vy
	vz = sim.particles[-1].vz
	df_icrf.append({
			"Des'n": object_name,
			"epoch": epoch,
			"x": x, 
			"y": y, 
			"z": z, 
			"vx": vx,
			"vy": vy,
			"vz": vz
		})
df_icrf = pd.DataFrame(df_icrf)

Searching NASA Horizons for 'K15Bz3T'... 
Found: 802375 (2015 BT613) 
Searching NASA Horizons for 'K08GG6Z'... 


/home/lshen/workspace/rebound/rebound/horizons.py:184: RuntimeWarning: Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.
  warnings.warn("Warning: Mass cannot be retrieved from NASA HORIZONS. Set to 0.", RuntimeWarning)


Found: (2008 GZ166) 


In [36]:
vec_1 = np.array(df_icrf.drop(columns = ["Des'n", "epoch"]).iloc[0])
vec_2 = np.array(df_icrf.drop(columns = ["Des'n", "epoch"]).iloc[1])

In [ ]:
# Rotation matrix
deg_rot = 23.43929111 * np.pi / 180.0 # Ecliptic obliquity at J2000
matrix_data = [[1, 0, 0],
			   [0, np.cos(deg_rot), np.sin(deg_rot)],
			   [0, -np.sin(deg_rot), np.cos(deg_rot)]]
rot_mat = np.array(matrix_data)
vec_1_trans = rot_mat @ vec_1[:3]
print(vec_1_trans)
vec_2_trans = rot_mat @ vec_2[:3]
print(vec_2_trans)
vec_1_trans = rot_mat @ vec_1[3:]
print(vec_1_trans)
vec_2_trans = rot_mat @ vec_2[3:]
print(vec_2_trans)

[ 2.61022701  1.17324332 -0.45346503]
[-2.20584194  1.77588458 -0.89241673]
[-0.00369093  0.00959092  0.00034762]
[-0.00555053 -0.00847365  0.00121104]


In [29]:
df1

,Des'n,epoch,x,y,z,vx,vy,vz
0,K15Bz3T,2460200.5,2.618677,1.175319,-0.453679,-0.003695,0.009599,0.000348
1,K08GG6Z,2460200.5,-2.197392,1.777961,-0.892631,-0.005554,-0.008466,0.001211


In [ ]:
clones = 1
for name in list_of_sbodies:
	flag, epoch, x,y,z,vx,vy,vz, weights  = sbd.query_sb_from_jpl(des=name,clones=clones)
	df1 = []
	if(flag):
		print("queried %s and returned at epoch %f" % (name,epoch))
		print("cartesian heliocentric position (au), velocity (au/year)")
		print("best-fit orbit:")
		i=0
		print(6*"%15.8e " % (x[i],y[i],z[i],vx[i],vy[i],vz[i]))
		print("cloned orbits:")
		for j in range (1,clones):
			df1.append({
				"Des'n": name,
				"epoch": epoch,
				"x": x[j], 
				"y": y[j], 
				"z": z[j], 
				"vx": vx[j], 
				"vy": vy[j], 
				"vz": vz[j]
			})

queried K15Bz3T and returned at epoch 2459531.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
-1.19455255e+00 -2.73173806e+00  2.64594639e-01  3.37682985e+00 -1.28480801e+00 -5.12029913e-01 
cloned orbits:
queried K08GG6Z and returned at epoch 2458911.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 1.21885948e+00 -2.51607580e+00  9.03672040e-01  3.52096162e+00  1.27150297e+00  3.29221529e-01 
cloned orbits:
